# Импорт библиотек

In [1]:
# Импорт библиотеки для работы с json-файлами
import json

# импортируем библиотеку numpy и pandas
import pandas as pd
import numpy as np

# импортируем библиотеку для работы с картами
from keplergl import KeplerGl

# импортируем библиотеку для вычисление высоты
import srtm

In [2]:
# Использовать для вычисления высоты 
# elevation_data = srtm.get_data()
# print('CGN Airport elevation (meters):', elevation_data.get_elevation(52.274269, 104.274618))

## Шпаргалка

<img src="https://ночная.москва/wp-content/uploads/2021/03/geograficheskie-koordinaty-moskvy-rossiya.png" width="50%" height="50%">

## Координаты вершин полигона г. Иркутск

In [3]:
# Границы полигона Иркутска
up_left = [52.446093, 103.964495]
up_right = [52.446093, 104.802521]

down_left = [52.013882, 103.964495]
down_right = [52.013882, 104.802521]

### Создание `pd.DataFrame` - полигона г. Иркутск 

In [4]:
coordinates = []

coordinates.append(up_left)
coordinates.append(up_right)
coordinates.append(down_left)
coordinates.append(down_right)

headers = ['latitude', 'longitude']

df_coordinates = pd.DataFrame(coordinates, columns = headers)
df_coordinates

,latitude,longitude
0,52.446093,103.964495
1,52.446093,104.802521
2,52.013882,103.964495
3,52.013882,104.802521


## Генерация множества координат

In [5]:
generated_list_of_latitude = np.arange(down_left[0], up_left[0], 0.001)

generated_list_of_longitude = np.arange(up_left[1], up_right[1], 0.001)

## Создание `pd.DataFrame` с "заполнением" полигона г. Иркутск

In [6]:
%%time

# Создание пустого датафрейма
df_append = pd.DataFrame()

# Цикл по все показателям широты
for lat in range(len(generated_list_of_latitude)):
    # Создание словаря с данными. К каждой широте добавляется массив долготы
    data = {
        'latitude': generated_list_of_latitude[lat],
        'longitude': generated_list_of_longitude
    }
    
    df = pd.DataFrame(data)
    
    df_append = df_append.append(df)
    
df_append.shape

CPU times: user 264 ms, sys: 24.3 ms, total: 288 ms
Wall time: 288 ms


(363287, 2)

## Применение библиотеки для вычисления высоты по координатам

In [34]:
df_append.head()

,latitude,longitude
0,52.013882,103.964495
1,52.013882,103.965495
2,52.013882,103.966495
3,52.013882,103.967495
4,52.013882,103.968495


### Создание функции для определения высоты

In [21]:
def altitude_determinant(lat: float, lon: float) -> int:
    """
    Функция для определения высоты по координатам (широта, долгота)
    
    Принимает:
    lat: float - широта
    lon: float - долгота
    
    Возвращает:
    altitude - высота
    """
    elevation_data = srtm.get_data()
    altitude = elevation_data.get_elevation(lat, lon)
    return altitude

In [35]:
%%time

df_append['altitude'] = df_append.apply(lambda x: altitude_determinant(x['latitude'],x['longitude']), axis=1)
df_append.head()

In [38]:
df_append.to_csv('altitude_info.csv', index=False)

## Заполнение таблицы в PostgreSQL

In [39]:
from sqlalchemy import create_engine
import psycopg2

In [51]:
# from authentication import LOGIN, PASSWORD, HOST

In [63]:
from auth import LOGIN_PG, PASSWORD_PG, HOST_PG, DATABASE_PG

In [64]:
df = df_append.copy()
df.head()

,latitude,longitude,altitude
0,52.013882,103.964495,766
1,52.013882,103.965495,765
2,52.013882,103.966495,765
3,52.013882,103.967495,762
4,52.013882,103.968495,750


In [67]:
df['city_name'] = 'Irkutsk'
df.head()

,latitude,longitude,altitude,city_name
0,52.013882,103.964495,766,Irkutsk
1,52.013882,103.965495,765,Irkutsk
2,52.013882,103.966495,765,Irkutsk
3,52.013882,103.967495,762,Irkutsk
4,52.013882,103.968495,750,Irkutsk


In [68]:
def authorization_pg():
    """
    Создание коннектора к БД PostgreSQL
    """
    login_name = LOGIN_PG
    login_password = PASSWORD_PG
    IP = HOST_PG
    engine_str = f'postgresql://{login_name}:{login_password}@{IP}:5432/{DATABASE_PG}'
    engine = create_engine(engine_str)
    return engine

def generate():
    """
    Создания словаря для заполнения таблицы в БД PostgreSQL
    """
    d = {
        'city_name': df.city_name,
        'latitude': df.latitude,
        'longitude': df.longitude,
        'altitude': df.altitude
    }
    df_into = pd.DataFrame(data=d)
    return df_into

def into_pg_table(pg_table_name):
    """
    Помещение данных в БД PostgreSQL
    """
    dataframe = generate()
    connector = authorization_pg()
    dataframe.to_sql(
        name = pg_table_name,
        con = connector,
        chunksize = 10000,
        index = False,
        if_exists = 'append'
    )

In [70]:
%%time
into_pg_table('city.topographic_city')

CPU times: user 9.55 s, sys: 202 ms, total: 9.76 s
Wall time: 28.6 s


## Отображение на карте гео-информации

In [181]:
with open('config.json') as f:
    config = json.load(f)

map_1 = KeplerGl()
map_1.config = config

map_1.add_data(data=df_append, name='9673oreu')

map_1

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'prj4oj', 'type': '…